In [32]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import camelot
import PyPDF2
import os
import re

In [2]:
def contains(string:str, contains:list) -> bool:
    for cont in contains:
        if cont in string:
            return True
    
    return False

In [3]:
def classify_file(file_path):
    try:
        extension = os.path.splitext(file_path)[1]

        match extension:
            case ".pdf":
                with open(file_path, "rb") as file:
                    reader = PyPDF2.PdfReader(file)
                    content = reader.pages[0].extract_text()
            case ".xlsx":
                content = pd.read_excel(file_path).to_string()
            case ".xlsb":
                content = pd.read_excel(file_path, engine='pyxlsb').to_string()
            case _:
                return "OTHER"

        content = content.lower()
        if "wrapbook" in content:
            return "OTHER"
        elif "purchase order" in content:
            return "PO"
        elif "payroll" in content:
            return "PR"
        elif contains(content, ["hot budget", "film production cost summary"]):
            return "CS"
        else:
            return "OTHER"
    except:
        print("classification error at ", file_path)
        return "OTHER"

In [4]:
def get_dept_from_line(ln:int) -> str:
    try:
        ln = int(ln)
    except ValueError:
        return ln

    if ln in range(51):
        return "PRE-PRODUCTION | WRAP LABOR"
    elif ln in range(51, 101):
        return "SHOOTING LABOR"
    elif ln in range(101, 114):
        return "PRE-PRODUCTION | WRAP EXPENSES"
    elif ln in range(114, 140):
        return "LOCATION AND TRAVEL"
    elif ln in range(140,151):
        return "MAKEUP, WARDROBE, AND ANIMALS"
    elif ln in range(151, 168):
        return "STUDIO | STAGE RENTAL / EXPENSES"
    elif ln in range(168,181):
        return "ART DEPARTMENT LABOR"
    elif ln in range(181, 193):
        return "ART DEPARTMENT EXPENSES"
    elif ln in range(193, 211):
        return "EQUIPMENT COSTS"
    elif ln in range(211, 217):
        return "FILMSTOCK, DEVELOP AND PRINT"
    elif ln in range(217,227):
        return "MISCELLANEOUS"
    elif ln in range(227, 234):
        return "DIRECTOR | CREATIVE FEES"
    elif ln in range(234, 271):
        return "TALENT LABOR"
    elif ln in range(271, 277):
        return "TALENT EXPENSES"
    elif ln in range(277, 282):
        return "POST PRODUCTION LABOR"
    elif ln in range(282, 330):
        return "EDITORIAL | FINISHING | POST PRODUCTION"
    else:
        return "OTHER"

In [5]:
# def read_hot_budget_cs(path) -> pd.DataFrame:
#     for df in camelot.read_pdf(path)._tables:
#         df = df.df
#         if "ESTIMATED COST SUMMARY" in df.to_string():
#             _df = df.copy()
#             start_idx = df[_df.apply(lambda row: row.str.contains("ESTIMATED COST SUMMARY").any(), axis=1)].index.values[0] + 1
#             break
    
#     print(_df)
    
#     _df.drop(12, inplace=True)

#     _df.columns = HB_CS_COLS
#     _df.drop(columns=["drop"], inplace=True)
#     _df = _df.loc[start_idx:]

#     _df = _df.replace([r"CS\d+\b ", r".*\n", "\)", ","], "", regex=True).replace("\(", "-", regex=True)

#     _df.iloc[:, 1:] = _df.iloc[:, 1:].replace("", np.nan).astype(float)

#     # _df = _df.dropna(thresh=3).fillna(0.0) # if drop empty rows

#     return _df.reset_index(drop=True)

In [70]:
HB_CS_COLS = ["SECTION", "drop", "BID TOTALS", "ACTUAL", "VARIANCE"]

def read_hot_budget_cs(path) -> pd.DataFrame:
    _df = camelot.read_pdf(path)._tables[1].df.copy()
    
    _df.drop(12, inplace=True)

    _df.columns = HB_CS_COLS
    _df.drop(columns=["drop"], inplace=True)
    _df = _df.loc[1:]

    _df = _df.replace([r"CS\d+\b ", r".*\n", "\)", ","], "", regex=True).replace("\(", "-", regex=True)

    _df[_df.columns[1:]] = _df.iloc[:, 1:].replace("", np.nan).astype(float)

    _df.fillna(0.0, inplace=True) # if drop empty rows

    return _df.reset_index(drop=True)


def read_GetActual_cs(path) -> pd.DataFrame:
    with open(path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        content = reader.pages[0].extract_text()

    content = re.sub(r"\b[A-Z]\s|Bid Actual|\,|\)", "", content.replace("(", "-"))
    content = content[content.find("Pre-production | Wrap Labor"):content.find("\nGRAND TOTAL")].split("\n")

    _df = pd.DataFrame(columns=["SECTION", "BID TOTALS", "ACTUAL"])
    for line in content:
        _df.loc[len(_df)] = line.split("$")

    _df[["BID TOTALS", "ACTUAL"]] = _df[["BID TOTALS", "ACTUAL"]].astype(float)
    _df = _df.drop(_df[_df.SECTION.str.contains("SUB TOTAL")].index)

    _df["VARIANCE"] = _df.ACTUAL - _df["BID TOTALS"]

    return _df


def read_cost_summary(path) -> pd.DataFrame:
    with open(path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        content = reader.pages[0].extract_text()
    try:
        if "HOT BUDGET" in content:
            return read_hot_budget_cs(path)
        elif "Film Production Cost Summary" in content:
            return read_GetActual_cs(path)
    except:
        return pd.DataFrame()

In [71]:
read_cost_summary("data/JOB_ACTUALS/22003_EA/EA - ACTUAL - 2.23.22-2-4.pdf")

,SECTION,BID TOTALS,ACTUAL,VARIANCE
0,Pre-production | Wrap Labor,7229.25,5346.29,-1882.96
1,Shooting Labor,7576.40,8562.74,986.34
2,Pre-production | Wrap Expenses,228.00,0.00,-228.00
3,Location And Travel,11000.00,11860.80,860.80
4,Makeup Wardrobe And Animals,0.00,0.00,0.00
5,Studio | Stage Rental / Expenses,0.00,0.00,0.00
6,Art Department Labor,2100.00,2170.00,70.00
7,Art Department Expenses,1500.00,2597.21,1097.21
8,Equipment Costs,5950.00,6118.67,168.67
9,Filmstock Develop And Print,400.00,227.76,-172.24


In [7]:
PR_COLS = ['LINE', 'PAYEE', 'PO', 'F1', 'F2', 'DAYS', 'RATE', 'BASE', '1.5', '2', '3', 'TAXABLE', 'NON-TAX', 'TOTAL ST', 'TOTAL OT', 'ACTUAL', 'FRINGE 1', 'FRINGE 2', 'LINE DESCRIPTION']

def read_pdf_payroll(path) -> pd.DataFrame:
    _df = camelot.read_pdf(path)._tables[0].df.copy()
    
    _df.columns = PR_COLS
    _df = _df.iloc[1:].reset_index(drop=True).replace("", np.nan).dropna(how="all")

    _df.LINE.fillna(_df.PAYEE, inplace=True)
    _df[['LINE', 'PAYEE']] = _df.LINE.str.split(" ", n=1, expand=True)

    _df = _df.replace(["\)", ","], "", regex=True).replace("\(", "-", regex=True)
    _df.ACTUAL = _df.ACTUAL.astype(float)

    return _df

In [8]:

PO_COLS = ["LINE", "PAYEE", "PO", "DATE", "PAYID", "ACTUAL", "LINE DESCRIPTION"]

def read_pdf_purchase_order(path) -> pd.DataFrame:
    _df = camelot.read_pdf(path)._tables[0].df.copy()
    
    _df.columns = PO_COLS
    _df = _df.iloc[1:].reset_index(drop=True).replace("", np.nan).dropna(how="all")

    _df.LINE.fillna(_df.PAYEE, inplace=True)
    _df[['LINE', 'PAYEE']] = _df.LINE.str.split(" ", n=1, expand=True)

    _df.ACTUAL.fillna(_df["LINE DESCRIPTION"], inplace=True)
    _df[['ACTUAL', 'LINE DESCRIPTION']] = _df.ACTUAL.str.split(" ", n=1, expand=True)

    _df = _df.replace(["\)", ","], "", regex=True).replace("\(", "-", regex=True)
    _df.ACTUAL = _df.ACTUAL.astype(float)

    return _df

In [9]:
def read_sheet(path, extension) -> pd.DataFrame:
    match extension:
        case ".xlsx":
            _df = pd.read_excel(path, header=4)
        case ".xlsb":
            _df = pd.read_excel(path, engine='pyxlsb', header=4)
    
    _df = _df.replace(["\)", ","], "", regex=True).replace("\(", "-", regex=True)
    _df.ACTUAL = _df.ACTUAL.astype(float)
    if "RATE" in _df.columns:
        _df.RATE = _df.RATE.astype(float)
    
    return _df

In [10]:
def read_payroll(path) -> pd.DataFrame:
    extension = os.path.splitext(path)[1]

    if extension == ".pdf":
        return read_pdf_payroll(path)
    else:
        return read_sheet(path, extension)


def read_purchase_order(path) -> pd.DataFrame:
    extension = os.path.splitext(path)[1]

    if extension == ".pdf":
        return read_pdf_purchase_order(path)
    else:
        return read_sheet(path, extension)

In [11]:
start_dir = "data/JOB_ACTUALS"

payroll_dfs = []
cs_dfs = []


for directory in os.listdir(start_dir):
    dir_path = os.path.join(start_dir, directory)
    good = True
    dir_files = {}

    if os.path.isdir(dir_path):
        for file in os.listdir(dir_path):
            file_path = os.path.join(dir_path, file)
            _type = classify_file(file_path)
            if not _type in dir_files:
                dir_files[_type] = []
            
            dir_files[_type].append(file_path)
    
    for _type in ["PO", "PR", "CS"]:
        if not _type in dir_files:
            good = False
    
    if dir_files.get("PR"):
        for pr_path in dir_files.get("PR"):
            df = read_payroll(pr_path)
            df["PROJECT_NAME"] = directory
            payroll_dfs.append(df)
    
    if dir_files.get("CS"):
        for path in dir_files.get("CS"):
            df = read_cost_summary(path)
            cs_dfs.append(df)

    if good:
        print(directory)
        
    


22003_EA
22004_USPS
22007_JOEL
22005_CORCEPT


In [12]:
pr_combined = pd.concat(payroll_dfs)
pr_combined.RATE = pr_combined.RATE.astype(float)
pr_combined.DAYS = pr_combined.DAYS.astype(float)

pr_combined["EST"] = pr_combined.RATE * pr_combined.DAYS
pr_combined["VARIANCE"] = pr_combined.ACTUAL - pr_combined.EST
pr_combined["VAR_PCT"] = pr_combined.VARIANCE / pr_combined.EST * 100
pr_combined["SECTION"] = pr_combined.LINE.apply(get_dept_from_line)


In [13]:
pr_combined.PROJECT_NAME.unique()

array(['22003_EA', '22004_USPS', '22007_JOEL', '22005_CORCEPT'],
      dtype=object)

In [14]:
# fig, ax = plt.subplots(figsize=(10, 10))

# BY = "SECTION"
# FOR = "VARIANCE"

# pr_combined.groupby(BY).mean().sort_values(FOR, ascending=True).query("%s > 0" % FOR)[FOR].plot(kind="barh", ax=ax)

# ax.set_title("Payee's Going Over Budget")
# ax.set_ylabel("Payee")
# ax.set_xlabel("Amount Over Budget ($)")

In [67]:
cs_combined = pd.concat(cs_dfs)

cs_grouped = cs_combined.groupby("SECTION").mean().sort_values("VARIANCE", ascending=False).round(2)
cs_grouped["VAR_PCT"] = (cs_grouped.VARIANCE / (cs_grouped["BID TOTALS"] + 1e-3)).round(2) * 100

cs_grouped

,BID TOTALS,ACTUAL,VARIANCE,VAR_PCT
SECTION,,,,
Miscellaneous,2785.00,6212.77,3427.77,123.0
Location And Travel,13007.94,14751.14,1743.20,13.0
Equipment Costs,14632.79,16076.91,1444.12,10.0
Art Department Expenses,6878.30,7713.11,834.82,12.0
Studio | Stage Rental / Expenses,986.21,1337.37,351.16,36.0
Makeup Wardrobe And Animals,1903.45,2191.78,288.33,15.0
Director | Creative Fees,17786.21,17922.42,136.21,1.0
Production Fee,31983.67,32101.70,118.03,0.0
Talent Expenses,0.00,48.45,48.45,4845000.0
